In [1]:
import pandas as pd
import numpy as np
from gnnwr import models, datasets, utils
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv(r'C:\Users\Admin\OneDrive\Documents\GitHub\Housing-Price-Time-Series-Forecasting\Housing-Price-Time-Series-Forecasting\data\modified\hdb_transaction_coordinates_sora.csv')
data.replace({np.nan: None})
data['resale_price_per_sqm'] = data['resale_price']/data['floor_area_sqm']
data.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,Unnamed: 0_y,latitude,longitude,sora,resale_price_per_sqm
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,0,1.375097,103.837619,0.129019,4250.000000
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10,1,1.373922,103.855621,0.129019,4044.117647
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4,2,1.373549,103.838176,0.129019,4130.434783
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10,3,1.367761,103.855357,0.129019,4264.705882
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10,4,1.371626,103.857736,0.129019,4264.705882


In [13]:
df = data[['remaining_lease', 'sora', 'latitude', 'longitude', 'resale_price_per_sqm']]
df = df.sample(n=100)

train_set, val_set, test_set = datasets.init_dataset(data=df,
                                                                 test_ratio=0.2, valid_ratio=0.1,
                                                                 x_column=['remaining_lease', 'sora'], 
                                                                 y_column=['resale_price_per_sqm'],
                                                                 spatial_column=['latitude', 'longitude'])
gnnwr = models.GNNWR(train_dataset = train_set,
                     valid_dataset = val_set, 
                     test_dataset = test_set,
                     activate_func=nn.ReLU(), 
                     start_lr=0.1, 
                     optimizer='Adadelta',
                     model_name = "GNNWR_VER1",
                     model_save_path = "./gnnwr_models")

x_min:[49.          0.08452727];  x_max:[96.       0.38197]
y_min:[3074.76635514];  y_max:[9314.28571429]


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gnnwr\utils.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept = self.__fit.params[0]


In [7]:
gnnwr.add_graph()

Add Graph Successfully


In [8]:
gnnwr.run(max_epoch = 1000,early_stop=250,print_frequency = 500)

  0%|          | 0/1000 [00:00<?, ?it/s]

 50%|█████     | 501/1000 [00:26<00:58,  8.50it/s]


Epoch:  500
learning rate:  0.09733963460294015
Train Loss:  2501517.222222222
Train R2: -0.88238
Train RMSE: 1581.61853
Train AIC: 1268.06052
Train AICc: 1271.31641
Valid Loss:  931234.25
Valid R2: -0.73833 

Best R2: -0.72734 



100%|██████████| 1000/1000 [00:59<00:00, 16.80it/s]



Epoch:  1000
learning rate:  0.03263617175376001
Train Loss:  1445486.861111111
Train R2: -0.08772
Train RMSE: 1202.28394
Train AIC: 1228.57203
Train AICc: 1231.66602
Valid Loss:  180860.625
Valid R2: 0.66239 

Best R2: 0.78748 

Best_r2: 0.7874767313376255


In [38]:
gnnwr.load_model('./gnnwr_models/GNNWR_VER1.pkl')

In [39]:
gnnwr.reg_result('./gnnwr_models/GNNWR_PM25_Result.csv')

,coef_remaining_lease,coef_sora,bias,Pred_resale_price_per_sqm,id
0,-912.877625,1979.869507,3023.630859,2654.595215,66
1,-695.997070,1417.241089,2753.535645,2652.043945,9
2,-708.217651,2312.974854,3239.360596,3603.229248,76
3,-738.028442,1951.184326,3037.310547,3323.550049,58
4,-859.555725,2934.961182,3552.770752,4442.064453,94
...,...,...,...,...,...
95,-847.270447,2702.491699,3428.211914,3049.644287,21
96,-759.566772,2390.109619,3272.767334,5081.081055,92
97,-857.591797,2752.012695,3453.454346,4260.755371,82
98,-874.012390,1741.592529,2900.364014,3628.152832,60


In [40]:
gnnwr.result()

--------------------Model Information-----------------
Model Name:           | GNNWR_VER1
independent variable: | ['remaining_lease', 'sora']
dependent variable:   | ['resale_price_per_sqm']

OLS coefficients: 
x0: 1065.31650
x1: 1572.80298
Intercept: 3585.57771

--------------------Result Information----------------
Test Loss: |             2810614.00000
Test R2  : |                  -1.59392
RMSE: |                     1676.48865
AIC:  |                      356.73581
AICc: |                      359.45862
F1:   |                        1.45968
F2:   |                       37.17972
f3_param_0: |                  0.17697
f3_param_1: |                  0.18662
f3_param_2: |                 20.24810


Visualisation

In [11]:
visualizer = utils.Visualize(data=gnnwr,lon_lat_columns=['longitude','latitude'])

In [12]:
visualizer.display_dataset(name='all',y_column='resale_price_per_sqm')

In [15]:
visualizer.display_dataset(name='train',y_column='resale_price_per_sqm')

In [16]:
visualizer.display_dataset(name='valid',y_column='resale_price_per_sqm')

In [17]:
visualizer.display_dataset(name='test',y_column='resale_price_per_sqm')


In [19]:
visualizer.coefs_heatmap('coef_remaining_lease')

In [20]:
visualizer.coefs_heatmap('coef_sora')

In [21]:
visualizer.coefs_heatmap('bias')